-Training completo tra 5mln a 10mln di TimeStamp e tra 5000 e 10000 episodi

-Per un tuning rapido da 500k a 1mln di TimeStamp e tra 500 a 1k episodi per trial (consigliati 500 trial)

-Per i test preliminari 1mln di timestamp e 1k/2k episodi

- CHAT con search dice che per il train vanno bene anche 1mln di timestamp

In [24]:
import tensorflow as tf
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
import numpy as np
import matplotlib.pyplot as plt
import os


from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import HParam
from stable_baselines3.common.callbacks import EvalCallback, CallbackList
from stable_baselines3.common.evaluation import evaluate_policy

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human'

SAC_1 -> [I 2025-02-16 08:35:02,806] Trial 6 finished with value: 2336.717724201964 and parameters: {'reset_noise_scale': 0.02535169706650553, 'forward_reward_weight': 1.6947930083902139, 'ctrl_cost_weight': 1.3382414657833144, 'healthy_reward': 2.454361643694453, 'contact_cost_weight': 9.801460983142638e-05, 'healthy_z_lower': 0.26138660668179525, 'healthy_z_upper': 1.2798527262125532, 'contact_force_min': -1.2175000215851723, 'contact_force_max': 0.8378346397233374, 'learning_rate': 0.0001093423040329136, 'batch_size': 256, 'gamma': 0.9586862210519205, 'tau': 0.03821261407257494, 'ent_coef': 0.01806334596909146, 'train_freq': 2, 'gradient_steps': 6}. Best is trial 6 with value: 2336.717724201964.


[I 2025-02-17 15:01:06,047] Trial 131 finished with value: 2877.213708915524 and parameters: {'reset_noise_scale': 0.18076079550448898, 'forward_reward_weight': 1.6236656710889608, 'ctrl_cost_weight': 1.2650706815726114, 'healthy_reward': 2.40623966986497, 'contact_cost_weight': 9.78401407249111e-06, 'healthy_z_lower': 0.17310762633784954, 'healthy_z_upper': 1.3741885293867462, 'contact_force_min': -1.2191856901258815, 'contact_force_max': 0.8954759302944116, 'learning_rate': 0.0008510732994554318, 'learning_starts': 10000, 'batch_size': 256, 'gamma': 0.9520212353011819, 'tau': 0.024539835196007352, 'ent_coef': 0.0139500037156449, 'train_freq': 3, 'gradient_steps': 6}. Best is trial 131 with value: 2877.213708915524.

SAC_2 -> [I 2025-02-17 18:33:32,594] Trial 234 finished with value: 2921.6025903890572 and parameters: {'reset_noise_scale': 0.20141630762026747, 'forward_reward_weight': 1.6278491287471968, 'ctrl_cost_weight': 1.2763028508139123, 'healthy_reward': 2.399103414226093, 'contact_cost_weight': 5.2247028604018e-06, 'healthy_z_lower': 0.16940418341206293, 'healthy_z_upper': 1.3272275689129294, 'contact_force_min': -1.2110815310071597, 'contact_force_max': 0.8982646857233315, 'learning_rate': 0.0006839314025125863, 'learning_starts': 4000, 'batch_size': 512, 'gamma': 0.9508951442412019, 'tau': 0.019742481119711143, 'ent_coef': 0.012500206557457209, 'train_freq': 3, 'gradient_steps': 7}. Best is trial 234 with value: 2921.6025903890572.


[I 2025-02-17 19:58:58,563] Trial 266 finished with value: 2606.9031733902284 and parameters: {'reset_noise_scale': 0.19226821652309825, 'forward_reward_weight': 1.6252187818921442, 'ctrl_cost_weight': 1.3518269064593467, 'healthy_reward': 2.404837557170063, 'contact_cost_weight': 7.933389808519208e-05, 'healthy_z_lower': 0.1683208288680806, 'healthy_z_upper': 1.3528414909566904, 'contact_force_min': -1.2353928919135455, 'contact_force_max': 0.888390169403758, 'learning_rate': 0.0008018300610795172, 'learning_starts': 10000, 'batch_size': 512, 'gamma': 0.9505557781892291, 'tau': 0.021876855067651642, 'ent_coef': 0.04252279860783029, 'train_freq': 3, 'gradient_steps': 6}. Best is trial 234 with value: 2921.6025903890572.


In [25]:
# Ipreparametri dell'envrionment
hp_reset_noise_scale= 0.20141630762026747 # scala del rumore quando l'ambiente viene resettato 
hp_forward_reward_weight = 1.6278491287471968 # peso del reward per il movimento in avanti
hp_ctrl_cost_weight = 1.2763028508139123 # peso del reward per il controllo
hp_healthy_reward = 2.399103414226093 # reward per la salute

hp_contact_cost_weight = 5.2247028604018e-06
healthy_z = (0.16940418341206293, 1.3272275689129294)
contact_force = (-1.2110815310071597, 0.8982646857233315)


# Iperparametri del modello/policy
hp_policy="MlpPolicy"           # Tipo di policy: una rete neurale MLP (Multilayer Perceptron) che mappa osservazioni ad azioni
hp_learning_rate=0.0006839314025125863          # Tasso di apprendimento: controlla la velocità con cui il modello apprende aggiornando i pesi
hp_learning_starts=4000           # Numero di passi di esplorazione prima di iniziare l'apprendimento
hp_batch_size=512                # Dimensione del batch per gli aggiornamenti stocastici: suddivide i dati raccolti nei mini-batch
hp_gamma=0.9508951442412019      # Fattore di sconto: determina l'importanza delle ricompense future rispetto a quelle immediate
hp_tau=0.019742481119711143      # Tasso di aggiornamento del target: controlla la velocità con cui i pesi del target network vengono aggiornati
hp_ent_coef=0.012500206557457209                 # Coefficiente di entropia: controlla l'incentivo all'esplorazione; 0 significa nessun bonus per l'entropia
hp_train_freq=3             # Frequenza di aggiornamento della policy: controlla la frequenza con cui la policy viene aggiornata
hp_gradient_steps=7         # Numero di passaggi di ottimizzazione per aggiornare la policy: controlla la quantità di aggiornamenti della policy


In [26]:
def make_env():
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium.

    Questa funzione istanzia l'ambiente "Ant-v5", uno degli ambienti recenti e ben supportati
    in Gymnasium. I parametri usati sono:
    - reset_noise_scale (0.1): determina la scala del rumore quando l'ambiente viene resettato.
    - render_mode ('None'): indica che non verrà effettuato il rendering durante l'esecuzione.

    Ritorna:
        gym.Env: l'ambiente Ant-v5 inizializzato.
    """
    
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=hp_reset_noise_scale, # scala del rumore quando l'ambiente viene resettato 
                    forward_reward_weight=hp_forward_reward_weight, # peso del reward per il movimento in avanti
                    ctrl_cost_weight=hp_ctrl_cost_weight, # peso del reward per il controllo
                    healthy_reward =hp_healthy_reward, # reward per la salute
                    contact_cost_weight=hp_contact_cost_weight,
                    healthy_z_range=healthy_z,
                    contact_force_range=contact_force,
                    render_mode='none')

In [27]:
# 1. Creiamo un ambiente vettorializzato (Vectorized Environment)
# Utilizziamo DummyVecEnv per gestire più istanze dell'ambiente come se fossero una singola entità.
# Qui passiamo la funzione make_env (definita in un'altra cella) che crea l'ambiente "Ant-v5".
#env = DummyVecEnv([make_env])  


NUM_ENVS=8
env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])


# 2. Normalizziamo osservazioni (obs) e ricompense (reward)
# VecNormalize scala le osservazioni e le ricompense per stabilizzare l'allenamento.
# Parametri:
#   norm_obs=True   -> Abilita la normalizzazione delle osservazioni.
#   norm_reward=True -> Abilita la normalizzazione delle ricompense.
#   clip_obs=10.     -> Limita i valori normalizzati dell'osservazione a un range [-10, 10] per evitare estremi.
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human'

In [28]:
# def linear_schedule(initial_value):
#     def func(progress_remaining):
#         return progress_remaining * initial_value
#     return func

In [29]:
# Crea ed allena il modello SAC con i parametri ottimizzati
model = SAC("MlpPolicy", env,
            learning_rate=hp_learning_rate,
            learning_starts=hp_learning_starts,
            buffer_size=50000,
            batch_size=hp_batch_size,
            gamma=hp_gamma,
            tau=hp_tau,
            ent_coef=hp_ent_coef,
            train_freq=hp_train_freq,
            gradient_steps=hp_gradient_steps,
            # seed=42,
            verbose=0,
            tensorboard_log="./sac_Ant_tensorboard/",  # Cartella per salvare i log di TensorBoard
            device='mps'                    # Specifica l'uso della GPU su Apple Silicon
            )

In [30]:
#eval_env = DummyVecEnv([make_env])

eval_env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])

eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=True, clip_obs=10.)

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human', 'rgb_array', 'depth_array']).
  logger.warn(
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:734: UserWarning: WARN: The environment is being initialised with render_mode='none' that is not in the possible render_modes (['human'

In [31]:
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs/best_model",
    log_path="./logs/",
    eval_freq=50000,
    deterministic=True,
    render=False
)

In [32]:
# 4. Alleniamo il modello
# Il parametro total_timesteps indica il numero totale di iterazioni (o passi)
# che il modello eseguirà durante l'allenamento. Ogni timestep rappresenta un'interazione
# con l'ambiente in cui il modello esegue un'azione e riceve un feedback, che viene poi
# usato per aggiornare la sua politica interna.
total_timesteps = 2000000  # Puoi aumentare questo valore per permettere al modello di acquisire più esperienza.
model.learn(total_timesteps=total_timesteps, callback=eval_callback)

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=400000, episode_reward=47.76 +/- 24.22
Episode length: 801.60 +/- 396.80
New best mean reward!
Eval num_timesteps=800000, episode_reward=69.76 +/- 2.46
Episode length: 1000.00 +/- 0.00
New best mean reward!
Eval num_timesteps=1200000, episode_reward=74.46 +/- 2.71
Episode length: 1000.00 +/- 0.00
New best mean reward!
Eval num_timesteps=1600000, episode_reward=70.76 +/- 10.35
Episode length: 1000.00 +/- 0.00
Eval num_timesteps=2000000, episode_reward=76.01 +/- 2.17
Episode length: 1000.00 +/- 0.00
New best mean reward!


In [33]:
#5. Salviamo il modello
model.save("sac_Ant_model")
env.save("vecnormalize_Ant.pkl")  # salviamo anche i parametri di normalizzazione

In [34]:
# 200-400 episodi sono adeguati 
# def evaluate_policy(env, policy, episodes=500):
#     """
#     Valuta una policy addestrata su un ambiente dato.

#     Parametri:
#     - env: L'ambiente di simulazione.
#     - policy: La policy addestrata da valutare.
#     - episodes: Numero di episodi da eseguire per la valutazione.

#     Ritorna:
#     - La ricompensa media e la deviazione standard delle ricompense ottenute.
#     """
#     total_rewards = []
#     for _ in range(episodes):
#         obs = env.reset()  # Reset dell'ambiente per iniziare un nuovo episodio
#         done = False
#         total_reward = 0
#         while not done:
#             action, _ = policy.predict(obs)  # Predice l'azione da eseguire
#             obs, reward, done, _ = env.step(action)  # Esegue l'azione e ottiene il feedback dall'ambiente
#             total_reward += reward  # Accumula la ricompensa ottenuta
#         total_rewards.append(total_reward)  # Aggiunge la ricompensa totale dell'episodio alla lista
#     return np.mean(total_rewards), np.std(total_rewards)  # Calcola e ritorna la media e la deviazione standard delle ricompense

# 200-400 episodi sono adeguati 
def evaluate_random_policy(env, episodes=500):
    """
    Valuta una policy casuale su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()
        done = [False] * env.num_envs
        episode_rewards = np.zeros(env.num_envs)
        while not all(done):
            actions = [env.action_space.sample() for _ in range(env.num_envs)]
            obs, rewards, done, infos = env.step(actions)
            episode_rewards += rewards
        total_rewards.extend(episode_rewards)
    mean_reward_random = np.mean(total_rewards)
    # std_reward_random = np.std(total_rewards)
    return mean_reward_random

In [35]:
# env.training = False # Setta l'environment in modalità di valutazione
# env.norm_reward = False # Disabilita la normalizzazione della reward. Questo è importante per valutare correttamente il modello.

# # Valutazione dopo l'addestramento
# mean_reward_trained, std_reward_trained = evaluate_policy(model, env, n_eval_episodes=500)  # Valuta la policy addestrata
# mean_reward_random, std_reward_random = evaluate_random_policy(env)  # Valuta la policy casuale

# # Stampa dei risultati
# print(f"Trained Policy: Mean Reward: {mean_reward_trained}")
# print(f"Random Policy: Mean Reward: {mean_reward_random}")

# # Creazione del grafico di confronto
# # labels = ['Random Policy', 'Trained Policy']
# # means = [mean_reward_random, mean_reward_trained]
# # stds = [std_reward_random, std_reward_trained]

# # plt.figure(figsize=(8, 5))
# # plt.bar(labels, means, yerr=stds, capsize=10, color=['skyblue', 'lightgreen'])
# # plt.ylabel('Mean Episodic Reward')
# # plt.title('Policy Comparison')
# # plt.show()